<a href="https://colab.research.google.com/github/kahram-y/first-repository/blob/master/etc/stat2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import norm

In [49]:
from google.colab import files
uploaded = files.upload()

Saving WA_Fn-UseC_-Telco-Customer-Churn.csv to WA_Fn-UseC_-Telco-Customer-Churn (1).csv


In [50]:
data = pd.read_csv('/content/WA_Fn-UseC_-Telco-Customer-Churn.csv')
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


# 이표본 t검정
이표본t검정은 두 그룹 간의 연속형 변수 평균에 차이가 있는지를 검정합니다. 데이터에서 고객 이탈 여부(Churn)에 따라 수치형 데이터의 평균에 차이가 있는지 가설을 세우고 검정해봅시다.

* 귀무가설(H₀): Churn 여부에 따른 MonthlyCharges 평균은 차이가 없다.
(𝜇이탈=𝜇유지)
* 대립가설(H₁): Churn 여부에 따른 MonthlyCharges 평균은 차이가 있다.
(𝜇이탈≠𝜇유지)

In [51]:
# 고객 이탈 여부(Churn)에 따라 두 그룹으로 분리
churn_yes = data.loc[data['Churn'] == 'Yes', 'MonthlyCharges']
churn_no = data.loc[data['Churn'] == 'No', 'MonthlyCharges']

In [52]:
# Kolmogorov–Smirnov 정규성 검정
# 평균과 표준편차를 이용해 정규분포로 표준화한 후 검정
churn_yes_z = (churn_yes - np.mean(churn_yes)) / np.std(churn_yes, ddof=1)
churn_no_z = (churn_no - np.mean(churn_no)) / np.std(churn_no, ddof=1)

ks_yes = stats.kstest(churn_yes_z, 'norm')
ks_no = stats.kstest(churn_no_z, 'norm')

print("Kolmogorov–Smirnov 정규성 검정 결과")
print("이탈 그룹 (Churn=Yes):", ks_yes)
print("유지 그룹 (Churn=No):", ks_no)

# 해석 기준
alpha = 0.05
if ks_yes.pvalue < alpha:
    print("\n→ 이탈 그룹은 정규분포를 따르지 않습니다. (p < 0.05)")
else:
    print("\n→ 이탈 그룹은 정규분포를 따른다고 볼 수 있습니다. (p ≥ 0.05)")

if ks_no.pvalue < alpha:
    print("→ 유지 그룹은 정규분포를 따르지 않습니다. (p < 0.05)")
else:
    print("→ 유지 그룹은 정규분포를 따른다고 볼 수 있습니다. (p ≥ 0.05)")

Kolmogorov–Smirnov 정규성 검정 결과
이탈 그룹 (Churn=Yes): KstestResult(statistic=np.float64(0.12673362560606677), pvalue=np.float64(1.257278559171083e-26), statistic_location=np.float64(-0.22262711449997571), statistic_sign=np.int8(-1))
유지 그룹 (Churn=No): KstestResult(statistic=np.float64(0.14979764830534703), pvalue=np.float64(8.102350503254542e-102), statistic_location=np.float64(-1.1197220501054292), statistic_sign=np.int8(1))

→ 이탈 그룹은 정규분포를 따르지 않습니다. (p < 0.05)
→ 유지 그룹은 정규분포를 따르지 않습니다. (p < 0.05)


In [53]:
# 정규성 가정이 깨졌을 때: Mann–Whitney U Test (비모수 검정)
u_stat, p_value = stats.mannwhitneyu(churn_yes, churn_no, alternative='two-sided')

print(f"U 통계량 = {u_stat:.4f}")
print(f"p-값 = {p_value:.4f}")

if p_value < 0.05:
    print("→ 두 그룹 간 MonthlyCharges의 중앙값에 유의한 차이가 있습니다.")
else:
    print("→ 두 그룹 간 MonthlyCharges의 중앙값에 유의한 차이가 없습니다.")

U 통계량 = 6003125.5000
p-값 = 0.0000
→ 두 그룹 간 MonthlyCharges의 중앙값에 유의한 차이가 있습니다.


# ANOVA 분산분석
세 개의 범주형 변수(MultipleLines, InternetService, Contract) 간의 연속형 변수 MonthlyCharges 평균이 서로 다른지를 ANOVA로 검정해봅니다.

* 귀무가설(H₀): 그룹별 MonthlyCharges 평균은 모두 같다.
* 대립가설(H₁): 적어도 한 그룹의 평균이 다르다.

In [62]:
# 1️⃣ 범주형 변수 중 범주가 3개 이상인 컬럼 찾기
cat_cols = data.select_dtypes(include='object').columns
cat_cols_over3 = [col for col in cat_cols if data[col].nunique() >= 3]

# 2️⃣ 연속형(수치형) 변수 찾기
num_cols = data.select_dtypes(include=np.number).columns

# 3️⃣ 결과 출력
print("📊 범주가 3개 이상인 범주형 변수:")
for col in cat_cols_over3:
    print(f"  - {col} ({data[col].nunique()}개 범주)")

print("\n📈 연속형(수치형) 변수:")
for col in num_cols:
    print(f"  - {col}")

📊 범주가 3개 이상인 범주형 변수:
  - customerID (7043개 범주)
  - MultipleLines (3개 범주)
  - InternetService (3개 범주)
  - OnlineSecurity (3개 범주)
  - OnlineBackup (3개 범주)
  - DeviceProtection (3개 범주)
  - TechSupport (3개 범주)
  - StreamingTV (3개 범주)
  - StreamingMovies (3개 범주)
  - Contract (3개 범주)
  - PaymentMethod (4개 범주)
  - TotalCharges (6531개 범주)

📈 연속형(수치형) 변수:
  - SeniorCitizen
  - tenure
  - MonthlyCharges


In [54]:
# 범주형 변수 리스트
cat_vars = ['MultipleLines', 'InternetService', 'Contract']

In [55]:
# 각 범주형 변수에 대해 그룹별 Kolmogorov–Smirnov 정규성 검정 수행
for var in cat_vars:
    print(f"\n 변수: {var}")

    for name, group in data.groupby(var):
        # MonthlyCharges 값 표준화 (z-score)
        charges_z = (group['MonthlyCharges'] - np.mean(group['MonthlyCharges'])) / np.std(group['MonthlyCharges'], ddof=1)

        # K-S 정규성 검정
        ks_stat, p_value = stats.kstest(charges_z, 'norm')

        # 결과 출력
        print(f"그룹 '{name}': KS-통계량={ks_stat:.4f}, p-값={p_value:.4f}", end=' ')
        if p_value < 0.05:
            print("→ 정규분포 아님")
        else:
            print("→ 정규분포 가능")


 변수: MultipleLines
그룹 'No': KS-통계량=0.2242, p-값=0.0000 → 정규분포 아님
그룹 'No phone service': KS-통계량=0.0794, p-값=0.0003 → 정규분포 아님
그룹 'Yes': KS-통계량=0.1291, p-값=0.0000 → 정규분포 아님

 변수: InternetService
그룹 'DSL': KS-통계량=0.0509, p-값=0.0000 → 정규분포 아님
그룹 'Fiber optic': KS-통계량=0.0633, p-값=0.0000 → 정규분포 아님
그룹 'No': KS-통계량=0.2931, p-값=0.0000 → 정규분포 아님

 변수: Contract
그룹 'Month-to-month': KS-통계량=0.1125, p-값=0.0000 → 정규분포 아님
그룹 'One year': KS-통계량=0.1368, p-값=0.0000 → 정규분포 아님
그룹 'Two year': KS-통계량=0.2149, p-값=0.0000 → 정규분포 아님


In [59]:
# 각 범주형 변수에 대해 Kruskal-Wallis Test (비모수 검정) 수행
for var in cat_vars:
    # 각 그룹별 MonthlyCharges 데이터 추출
    groups = [group['MonthlyCharges'].values for name, group in data.groupby(var)]

    # Kruskal-Wallis H-test 수행
    h_stat, p_value = stats.kruskal(*groups)

    print(f"\n 변수: {var}")
    print(f"H-통계량 = {h_stat:.4f}, p-값 = {p_value:.4f}")

    # 결과 해석
    alpha = 0.05
    if p_value < alpha:
        print(f"→ 유의수준 {alpha}에서 귀무가설 기각: 그룹별 MonthlyCharges 중앙값에 유의한 차이가 있습니다.")
    else:
        print(f"→ 유의수준 {alpha}에서 귀무가설 채택: 그룹별 MonthlyCharges 중앙값 차이가 통계적으로 유의하지 않습니다.")


 변수: MultipleLines
H-통계량 = 1983.5366, p-값 = 0.0000
→ 유의수준 0.05에서 귀무가설 기각: 그룹별 MonthlyCharges 중앙값에 유의한 차이가 있습니다.

 변수: InternetService
H-통계량 = 5529.7336, p-값 = 0.0000
→ 유의수준 0.05에서 귀무가설 기각: 그룹별 MonthlyCharges 중앙값에 유의한 차이가 있습니다.

 변수: Contract
H-통계량 = 21.2262, p-값 = 0.0000
→ 유의수준 0.05에서 귀무가설 기각: 그룹별 MonthlyCharges 중앙값에 유의한 차이가 있습니다.


# 카이제곱 독립성 검정
gender와 Churn 변수의 관련성을 검정합니다.

* 귀무가설(H₀): gender와 Churn는 서로 독립이다. (즉, 성별은 고객 이탈 여부에 영향을 미치지 않는다)
* 대립가설(H₁): gender와 Churn는 서로 독립이 아니다. (즉, 성별은 고객 이탈 여부에 영향을 미친다)

In [61]:
# 분할표(crosstab) 생성
contingency_table = pd.crosstab(data['Partner'], data['Churn'])
print("분할표 (Contingency Table):")
print(contingency_table)

# 카이제곱 독립성 검정 수행
chi2_stat, p_value, dof, expected = stats.chi2_contingency(contingency_table)

print("\n카이제곱 검정 결과")
print(f"Chi2 통계량 = {chi2_stat:.4f}")
print(f"p-값 = {p_value:.4f}")
print(f"자유도 (degrees of freedom) = {dof}")
print("기대 빈도표 (Expected Frequencies):")
print(pd.DataFrame(expected, index=contingency_table.index, columns=contingency_table.columns))

# 결과 해석
alpha = 0.05
if p_value < alpha:
    print(f"\n→ 유의수준 {alpha}에서 귀무가설 기각: Partner 여부는 Churn과 관련이 있습니다.")
else:
    print(f"\n→ 유의수준 {alpha}에서 귀무가설 채택: Partner 여부는 Churn과 관련이 없다고 볼 수 있습니다.")

분할표 (Contingency Table):
Churn      No   Yes
Partner            
No       2441  1200
Yes      2733   669

카이제곱 검정 결과
Chi2 통계량 = 158.7334
p-값 = 0.0000
자유도 (degrees of freedom) = 1
기대 빈도표 (Expected Frequencies):
Churn           No       Yes
Partner                     
No       2674.7883  966.2117
Yes      2499.2117  902.7883

→ 유의수준 0.05에서 귀무가설 기각: Partner 여부는 Churn과 관련이 있습니다.
